### Import the libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Modelling
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

# Experiment tracking
import mlflow

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Set up experiment tracking with MLFlow

In [ ]:
# Store experiment files on the database
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Create a new experiment
mlflow.set_experiment("duration-trip")

### Helper functions

In [3]:
# Transform nyc-taxi dataset
def read_dataframe(file_path):

    # Load csv dataset
    df = pd.read_parquet(file_path)

    # Convert pick up and drop off location into string format
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    # Convert pick up and drop off time column into datetime format
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    # Add duration column, differences between pick up and drop off time
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    
    # Convert duration column into minute format
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # Filter duration between 1 and 60 minutes
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    return df

In [ ]:
# Load data train and validation
df_train_path = 'data/green_tripdata_2021-01.parquet'
df_val_path = 'data/green_tripdata_2021-02.parquet'

df_train = read_dataframe(df_train_path)
df_val = read_dataframe(df_val_path)

print(f'Dimension of data train : {df_train.shape[0]}, {df_train.shape[1]}')
print(f'Dimension of data validation : {df_val.shape[0]}, {df_val.shape[1]}')

### Data preparation

Feature engineering experimentation

In [12]:
# Feature engineering to concatenate pick up and drop off location id
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

Preparation for data train and data validation

In [14]:
# Define categorical and numerical columns
categorical = ['PU_DO']
numerical = ['trip_distance']

# Instantiate dictionary vectorizer
dv = DictVectorizer()

# Fit and transform data train into vectors with feature-value format
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Transform data validation into vectors based on pattern got from data train
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# Define target column for predicting
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

### Modelling with MLflow

#### 1. Experiment without MLflow

Linear regression

In [ ]:
# Linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

Lasso regression

In [ ]:
# Lasso regression
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

Ridge regression

In [ ]:
# Ridge regression
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

#### 2. Experiment tracking without parameter

Linear Regression

In [17]:
# Linear regression

with mlflow.start_run():
    
    # Log general info, ex : developer_name
    mlflow.set_tag("developer", "farel")
    mlflow.set_tag("model", "linear-regression")
    
    # Log parameters
    mlflow.log_param("train-data-path", df_train_path)
    mlflow.log_param("validation-data-path", df_val_path)
    
    # Training model
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    # mlflow.sklearn.log_model(lr, 'Linear Regression')
    
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    # Keep info related to metrics resulted from the model
    mlflow.log_metric("rmse", rmse)

Lasso Regression

In [18]:
# Lasso regression

with mlflow.start_run():
    
    # Log general info, ex : developer_name
    mlflow.set_tag("developer", "farel")
    mlflow.set_tag("model", "lasso-regression")
    
    # Log parameters
    mlflow.log_param("train-data-path", df_train_path)
    mlflow.log_param("validation-data-path", df_val_path)
    
    # Log model
    l1 = Lasso()
    l1.fit(X_train, y_train)
    # mlflow.sklearn.log_model(l1, 'Lasso Regression')
    
    y_pred = l1.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    # To keep info related to metrics resulted from the model
    mlflow.log_metric("rmse", rmse)

Ridge Regression

In [19]:
# Ridge regression

with mlflow.start_run():
    
    # Log general info, ex : developer_name
    mlflow.set_tag("developer", "farel")
    mlflow.set_tag("model", "ridge-regression")
    
    # Log parameters
    mlflow.log_param("train-data-path", df_train_path)
    mlflow.log_param("validation-data-path", df_val_path)
    
    # Log model
    l2 = Ridge()
    l2.fit(X_train, y_train)
    # mlflow.sklearn.log_model(l2, 'Ridge Regression')
    
    y_pred = l2.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    # To keep info related to metrics resulted from the model
    mlflow.log_metric("rmse", rmse)

#### 3. Experiment MLFlow with single parameter

Lasso Regression

In [23]:
# Lasso regression

with mlflow.start_run():
    
    # Log general info, ex : developer_name
    mlflow.set_tag("developer", "farel")
    mlflow.set_tag("model", "lasso-regression")
    
    # Log parameters
    mlflow.log_param("train-data-path", df_train_path)
    mlflow.log_param("validation-data-path", df_val_path)
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    # Training model
    l1 = Lasso(alpha=alpha)
    l1.fit(X_train, y_train)
    # mlflow.sklearn.log_model(l1, 'Lasso Regression')
    
    y_pred = l1.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    # Keep info related to metrics resulted from the model
    mlflow.log_metric("rmse", rmse)

Ridge Regression

In [24]:
# Ridge regression

with mlflow.start_run():
    
    # Log general info, ex : developer_name
    mlflow.set_tag("developer", "farel")
    mlflow.set_tag("model", "ridge-regression")
    
    # Log parameters
    mlflow.log_param("train-data-path", df_train_path)
    mlflow.log_param("validation-data-path", df_val_path)
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    # Training model
    l2 = Ridge(alpha=alpha)
    l2.fit(X_train, y_train)
    # mlflow.sklearn.log_model(l2, 'Ridge Regression')
    
    y_pred = l2.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    # Keep info related to metrics resulted from the model
    mlflow.log_metric("rmse", rmse)

#### 4. Experiment tracking with multiple parameters 

In [30]:
# Lasso regression
alphas = [0.1, 0.01, 0.001]
for alpha in alphas:
    with mlflow.start_run():
        # Log general info, ex : developer_name
        mlflow.set_tag("developer", "farel")
        mlflow.set_tag("model", "lasso-regression")
        mlflow.set_tag("experiment", "multiple-parameters")        

        # Log a parameter
        mlflow.log_param("train-data-path", df_train_path)
        mlflow.log_param("validation-data-path", df_val_path)
        mlflow.log_param("alpha", alpha)
        
        # Train model
        l1 = Lasso(alpha=alpha)
        l1.fit(X_train, y_train)
        # mlflow.sklearn.log_model(l1, 'Lasso Regression')
        
        # Evaluate model
        y_pred = l1.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)

        # Log metrics
        mlflow.log_metric("rmse", rmse)

Ridge Regression

In [31]:
# Ridge regression
alphas = [0.1, 0.01, 0.001]
for alpha in alphas:
    with mlflow.start_run():
        # Log general info, ex : developer_name
        mlflow.set_tag("developer", "farel")
        mlflow.set_tag("model", "ridge-regression")
        mlflow.set_tag("experiment", "multiple-parameters")        

        # Log a parameter
        mlflow.log_param("train-data-path", df_train_path)
        mlflow.log_param("validation-data-path", df_val_path)
        mlflow.log_param("alpha", alpha)
        
        # Train model
        l2 = Ridge(alpha=alpha)
        l2.fit(X_train, y_train)
        # mlflow.sklearn.log_model(l2, 'Ridge Regression')
        
        # Evaluate model
        y_pred = l2.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)

        # Log metrics
        mlflow.log_metric("rmse", rmse)

#### 5. XGBoost Model

Data Preparation

In [32]:
# Convert data train and validation into optimized DMatrix format
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [33]:
# Define objective function for training xgboost model using MLflow
def objective(params):
    with mlflow.start_run():
        
        # Log general info
        mlflow.set_tag("developer", "farel")
        mlflow.set_tag("model", "xgboost")
        mlflow.set_tag("experiment", "tuning-hyperparams")        

        # Log batch of parameters
        mlflow.log_params(params)

        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        # Evaluate model
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)

        # Log metrics
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [34]:
# Define hyperparameters combination
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

Training Model

In [ ]:
# Call objective function to running xgboost model
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials(),
    verbose=False
)

Find Best Parameters

In [ ]:
# Find best parameters from run experiment
best_run_id = mlflow.search_runs(
    filter_string="tags.model='xgboost' and tags.experiment='tuning-hyperparams'",
    order_by=["metrics.rmse"],
    max_results=1
).run_id

best_params = mlflow.get_run(best_run_id[0]).data.params
best_params


Training Model with Best Parameters

In [ ]:
# Do autologging using mlflow built-in function
mlflow.xgboost.autolog()

# Train model
booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)    